#Run cross-group analyses that lyman doesn't suppport

In [1]:
#os and i/o
import os
import numpy as np
import glob
from os.path import abspath
import csv

#scientific computing
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats, optimize
from pandas import DataFrame, Series
import seaborn as sns
import random as rd
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import scipy.stats

#ipython add-ons
from IPython.parallel import Client
from IPython.display import Image
import multiprocessing

##nipype
import nibabel as nib
from nipype.pipeline.engine import Node, MapNode, Workflow
from nipype.interfaces.io import DataGrabber, DataFinder, DataSink
from nipype.interfaces import fsl
from nipype.interfaces.fsl import ImageMeants
from nipype.interfaces.fsl import ImageStats

%matplotlib inline

In [2]:
#preliminary housekeeping
home_dir = '/data/home/iballard/fd/'
subj_file = home_dir + 'subjects.txt'
sub_list = list(np.loadtxt(subj_file,'string'))
os.chdir(home_dir)
exps = ['sim','ser']
runs = map(str,range(1,4))

In [3]:
group_dir = home_dir + 'analysis/group_ser_sim_4mm_PEmb/'
if not os.path.exists(group_dir):
    os.mkdir(group_dir)

#Make design file

In [82]:
#alternatively, make correlation design file
# cov = np.loadtxt(home_dir + '/analysis/omegas_powell.txt')
cov = np.loadtxt(home_dir + 'vta_mb.txt')
# cov = [x for x in cov if x> -5000]
print len(cov)
##make regressor and contrast inputs according to FSL rules for paired ttest
# regressors = dict(cov = list(cov) )
regressors = dict(cov = list(cov))
contrasts=[['ser-corr','T',['cov'],[1]]]
# regressors = dict(mean = list(np.ones(len(cov))))
# contrasts=[['ser-corr','T',['mean'],[1]]]

##set up WF so I can control where outputs go
model = Node(fsl.MultipleRegressDesign(regressors = regressors, contrasts = contrasts),name='model')
wf = Workflow(name = 'design',base_dir = group_dir)
sink = Node(DataSink(),name = 'sinker',base_directory = group_dir)
wf.connect(model,'design_con',sink,'output.@con')
wf.connect(model,'design_grp',sink,'output.@grp')
wf.connect(model,'design_mat',sink,'output.@mat')
wf.run()

INFO:workflow:['check', 'execution', 'logging']
INFO:workflow:Running serially.
INFO:workflow:Executing node model in dir: /data/home/iballard/fd/analysis/group_ser_sim_4mm_PEmb/design/model
INFO:workflow:Executing node sinker in dir: /data/home/iballard/fd/analysis/group_ser_sim_4mm_PEmb/design/sinker


32


#Merge files

In [102]:
#merge the 3d files from ffx analysis into concatenated 4d files
images = ['cope','varcope','dof','zstat']
ims_to_files = {'cope':'cope1.nii.gz', 'varcope':'varcope1.nii.gz',
                'dof':'tdof_t1.nii.gz','zstat':'zstat1.nii.gz'}
contrast = 'PE_state'
for f in images:
    out_dir = group_dir + contrast
    if not os.path.exists(out_dir):
        os.mkdir(out_dir)
    
    #add files to merge command
    out_f = out_dir + '/' + f + '_merged.nii.gz'
    if not os.path.exists(out_f):
        cmd_str = ['fslmerge','-t',out_f]

        for exp in ['ser','sim']: #order matters now
            for sub in sub_list:
                sub_f = home_dir + '/analysis/' + exp + '_4mm-PEfb-diff/' + sub + '/ffx/mni/smoothed/' + \
                    contrast + '/' + ims_to_files[f]
                cmd_str.append(sub_f)
        cmd_str = ' '.join(cmd_str)
        os.system(cmd_str)

In [64]:
#merge the 3d files from ffx analysis into concatenated 4d files
images = ['cope','varcope','dof','zstat']
ims_to_files = {'cope':'cope1.nii.gz', 'varcope':'varcope1.nii.gz',
                'dof':'tdof_t1.nii.gz','zstat':'zstat1.nii.gz'}
contrast = 'A'
subjects = list(np.loadtxt(home_dir+ '/subjects.txt',str))
# bad_subs = ['fd_104' ,'fd_107', 'fd_108' ,'fd_109' ,'fd_115' ,'fd_122','fd_133' ,'fd_141']
bad_subs = []
behav_list = [s for s in subjects if s not in bad_subs]
in_tuples = [('A_v_all','ser','PEfb-diff'),('PE_mf','ser','PEfb-diff'),
             ('PE_mf_state','ser','PEfb-diff'),('PE_mb','ser','PEfb-diff'),
            ('PE_mb_state','ser','PEfb-diff'),('PE','ser','PEfb'),
            ('state','ser','PEfb'),('PE_state','ser','PEfb')]

for f in images:
    out_dir = group_dir + contrast
    if not os.path.exists(out_dir):
        os.mkdir(out_dir)
    
    #add files to merge command
    for exp in ['ser']: #order matters now
        out_f = out_dir + '/' + f + '_' + exp + '_merged.nii.gz'
        if not os.path.exists(out_f):
            cmd_str = ['fslmerge','-t',out_f]
            for sub in behav_list:
                sub_f = home_dir + '/analysis/' + exp + '_4mm-PEfb/' + sub + '/ffx/mni/smoothed/' + \
                    contrast + '/' + ims_to_files[f]
                cmd_str.append(sub_f)
        cmd_str = ' '.join(cmd_str)
        os.system(cmd_str)

In [17]:
#Perform FFX analysis for comparing between experiments
images = ['cope']
ims_to_files = {'cope':'cope1.nii.gz'}
subjects = list(np.loadtxt(home_dir+ '/subjects.txt',str))
# bad_subs = ['fd_122']
# behav_list = [s for s in subjects if s not in bad_subs]

contrast = 'PE_mf_state'
if not os.path.exists(group_dir + contrast):
    os.mkdir(group_dir + contrast)

for f in images:
    out_dir = group_dir + contrast + '/diff'
    if not os.path.exists(out_dir):
        os.mkdir(out_dir)
    
    #add files to merge command
    out_f = out_dir + '/' + f + '_' + 'ser-sim' + '_merged.nii.gz'
    if not os.path.exists(out_f):
        merge_str = ['fslmerge','-t',out_f]
        for sub in subjects:

            cmd_str = ['fslmaths']
            
            sub_dir = home_dir + '/analysis/ser_4mm-PEfb-diff/' + sub + '/ffx/mni/smoothed/'
            sub_f = sub_dir + contrast + '/' + ims_to_files[f]
            cmd_str.append(sub_f)
            
            cmd_str.append('-sub')
           
            sub_dir = home_dir + '/analysis/sim_4mm-PEfb-diff/' + sub + '/ffx/mni/smoothed/'
            sub_f = sub_dir + contrast + '/' + ims_to_files[f]
            cmd_str.append(sub_f)

            
            out_file = out_dir + '/diff_' + sub + '.nii.gz'
            cmd_str.append(out_file)
            os.system(' '.join(cmd_str))

            merge_str.append(out_file)
        merge_str = ' '.join(merge_str)
        os.system(merge_str)

In [12]:
# #Normalize mb images by mf images
# images = ['cope']
# ims_to_files = {'cope':'cope1.nii.gz'}
# subjects = list(np.loadtxt(home_dir+ '/subjects.txt',str))
# bad_subs = ['fd_104' ,'fd_107', 'fd_108' ,'fd_109' ,'fd_115' ,'fd_133' ,'fd_141']
# # bad_subs = ['fd_122']
# behav_list = [s for s in subjects if s not in bad_subs]

# contrast = 'state'

# for f in images:
#     out_dir = group_dir + contrast + '/norm'
#     if not os.path.exists(out_dir):
#         os.mkdir(out_dir)
    
#     #add files to merge command
#     for exp in ['ser']:#,'sim']: #order matters now
#         out_f = out_dir + '/' + f + '_' + exp + '_merged.nii.gz'
#         if not os.path.exists(out_f):
#             merge_str = ['fslmerge','-t',out_f]
#             for sub in behav_list:
#                 cmd_str = ['fslmaths']
#                 sub_dir = home_dir + '/analysis/' + exp + '_4mm-PEfb-diff/' + sub + '/ffx/mni/smoothed/'
#                 sub_f = sub_dir + contrast + '/' + ims_to_files[f]
#                 cmd_str.append(sub_f)
#                 cmd_str.append('-div')
#                 sub_f = sub_dir + 'PE_mf' + '/' + ims_to_files[f]     
#                 cmd_str.append(sub_f)
#                 out_file = out_dir + '/norm_' + sub + '_' + exp + '.nii.gz'
#                 cmd_str.append(out_file)
#                 os.system(' '.join(cmd_str))

#                 merge_str.append(out_file)
                
#         merge_str = ' '.join(merge_str)
#         os.system(merge_str)

#Do fitting with randomise

In [18]:
exp = 'ser'
contrast = 'PE_mf_state'
im_dir = group_dir + contrast +'/diff/'
stat_file = im_dir + 'cope_ser-sim_merged.nii.gz'
mask_file = home_dir + '/analysis/ser_4mm-PEfb-diff/group/mni/' + contrast + '/mask.nii.gz'

cmd_str = ['randomise','-i',stat_file,'-o',im_dir,'-m',mask_file,'-T','-1','-n','1000']
cmd_str = ' '.join(cmd_str)
os.system(cmd_str)
# print cmd_str

0

In [84]:
#do fitting with randomise
#input image files
def randomise(in_tuple):
    contrast,exp,altmodel = in_tuple

    im_dir = group_dir + contrast + '/'
    if not os.path.exists(im_dir):
        os.mkdir(im_dir)
        
#     im_dir = group_dir + 'PE_contrast +'/'
#     stat_file = im_dir + 'cope_' + exp + '_merged.nii.gz'
#     mask_file = home_dir + '/analysis/ser_4mm-' + altmodel + '/group/mni/' + contrast + '/mask.nii.gz'
#     cmd_str = ['randomise','-i',stat_file,'-o',im_dir,'-m', mask_file,'-C','2.3','-1','-n','1000']
    
    #input design files
    stat_file = home_dir + 'analysis/group_ser_sim_4mm/' + contrast + \
        '/cope_' + exp + '_merged.nii.gz'

    design_dir = group_dir + '/design/sinker/output/'
    t_con_file = design_dir + 'design.con'
    cov_split_file = design_dir + 'design.grp'
    design_file = design_dir + 'design.mat'
    cmd_str = ['randomise','-i',stat_file,'-o',im_dir,'-d',design_file,
               '-t',t_con_file,'-m',mask_file,'-n','1000','-D','-C','1.7']
    cmd_str = ' '.join(cmd_str)
    os.system(cmd_str)

In [85]:
in_tuples = [('A_v_all','ser','PEfb-diff'),('PE_mf','ser','PEfb-diff'),
             ('PE_mf_state','ser','PEfb-diff'),('PE_mb','ser','PEfb-diff'),
            ('PE_mb_state','ser','PEfb-diff'),('PE','ser','PEfb'),
            ('state','ser','PEfb'),('PE_state','ser','PEfb'),
            ('A','ser','PEfb-diff')]
pool = multiprocessing.Pool(processes = len(in_tuples))
pool.map(randomise,in_tuples)
pool.terminate()
pool.join()

#Flameo

In [ ]:
#Make design file for paired t-test
num_subs = len(sub_list)
group_dir = home_dir + 'analysis/group_ser_sim_4mm/'

##make regressor and contrast inputs according to FSL rules for paired ttest
regressors = dict(diff = list(np.hstack([np.ones(num_subs)*1.0,np.ones(num_subs)-2.0])))
for i in range(num_subs):
    col = np.zeros(num_subs*2)
    col[i] = 1.0
    col[i + num_subs] = 1.0
    regressors['sub' + str(i)] = list(col)
    
# contrasts=[['ser-sim','T',['diff'],[1]], ['sim-ser','T',['diff'],[-1]]]
contrasts=[['ser-sim','T',['diff'],[1]]]

##set up WF so I can control where outputs go
# model = Node(fsl.MultipleRegressDesign(regressors = regressors, contrasts = contrasts),name='model')

#for randomise, we need groups for exchangability blocks
group = list(np.hstack([np.arange(num_subs)+1,np.arange(num_subs)+1]))
model = Node(fsl.MultipleRegressDesign(regressors = regressors, 
                                       contrasts = contrasts, groups = group),name='model')


wf = Workflow(name = 'design',base_dir = group_dir)
sink = Node(DataSink(),name = 'sinker',base_directory = group_dir)
wf.connect(model,'design_con',sink,'output.@con')
wf.connect(model,'design_grp',sink,'output.@grp')
wf.connect(model,'design_mat',sink,'output.@mat')
wf.run()

In [ ]:
# #Run flameo
# contrast = 'PE_mb'
# wf = Workflow(name = 'flameo',base_dir = group_dir)
# exp = 'ser'
# #input image files
# im_dir = group_dir + contrast
# cope_file = im_dir + '/cope_' + exp + '_merged.nii.gz'
# var_cope_file = im_dir + '/varcope_' + exp + '_merged.nii.gz'
# dof_var_cope_file = im_dir + '/dof_' + exp + '_merged.nii.gz'

# #input design files
# design_dir = group_dir + '/design/sinker/output/'
# t_con_file = design_dir + 'design.con'
# cov_split_file = design_dir + 'design.grp'
# design_file = design_dir + 'design.mat'

# #mask file
# mask_file = home_dir + '/analysis/ser_8mm-PEfb-diff/group/mni/' + contrast + '/mask.nii.gz'

# #set up flame
# flameo = Node(fsl.FLAMEO(cope_file = cope_file, var_cope_file= var_cope_file,
#                     dof_var_cope_file = dof_var_cope_file, run_mode = 'flame1',
#                    t_con_file = t_con_file, cov_split_file = cov_split_file, 
#                     design_file = design_file, mask_file = mask_file), name = 'flameo')

# #estimate smoothness
# smoothest = Node(fsl.SmoothEstimate(mask_file = mask_file),name='smoothest')
# wf.connect(flameo,'zstats',smoothest,'zstat_file')

# #set up cluster correction
# cluster = Node(fsl.Cluster(threshold = 2.3, pthreshold = 0.05, out_threshold_file = True,
#                           out_index_file = True, out_localmax_txt_file=True,
#                            peak_distance=30, use_mm=True),name='cluster')
# wf.connect(smoothest,'dlh',cluster,'dlh')
# wf.connect(smoothest,'volume',cluster,'volume')
# wf.connect(flameo,'zstats',cluster,'in_file')

# ##set up DataSink so I can control where outputs go
# sink = Node(DataSink(),name = 'sinker',base_directory = group_dir)
# wf.connect(flameo,'copes',sink,'output.copes')
# wf.connect(flameo,'fstats',sink,'output.fstats')
# wf.connect(flameo,'mrefvars',sink,'output.mrefvars')
# wf.connect(flameo,'pes',sink,'output.pes')
# wf.connect(flameo,'res4d',sink,'output.res4d')
# wf.connect(flameo,'stats_dir',sink,'output.stats_dir')
# wf.connect(flameo,'tdof',sink,'output.tdof')
# wf.connect(flameo,'tstats',sink,'output.tstats')
# wf.connect(flameo,'var_copes',sink,'output.varcopes')
# wf.connect(flameo,'weights',sink,'output.weights')
# wf.connect(flameo,'zfstats',sink,'output.zfstats')
# wf.connect(flameo,'zstats',sink,'output.zstats')

# wf.connect(cluster,'index_file',sink,'output.cluster.index_file')
# wf.connect(cluster,'localmax_txt_file',sink,'output.cluster.localmax_txt_file')
# wf.connect(cluster,'localmax_vol_file',sink,'output.cluster.localmax_vol_file')
# wf.connect(cluster,'max_file',sink,'output.cluster.max_file')
# wf.connect(cluster,'mean_file',sink,'output.cluster.mean_file')
# wf.connect(cluster,'pval_file',sink,'output.cluster.pval_file')
# wf.connect(cluster,'size_file',sink,'output.cluster.size_file')
# wf.connect(cluster,'threshold_file',sink,'output.cluster.threshold_file')

# wf.run()

In [ ]:
# wf.write_graph('workflow_graph.dot')
# Image(filename = '/data/home/iballard/fd/analysis/group_ser_sim/flameo/workflow_graph.dot.png' ) 